# 03_landing_convert_csv_to_parquet_job
---
Este notebook realiza a conversão dos arquivos csv presentes na camada **Stage** (exceto os chunks `flights_part_*`) para o formato **Parquet**, mantendo o schema original dos dados.


In [15]:
# Parameters

stage_path = "/opt/airflow/data-layer/stage"


In [16]:
import os
from pyspark.sql import SparkSession, DataFrame
from transformer.utils.spark_helpers import get_spark_session
from transformer.utils.file_io import check_files_in_folder
from transformer.utils.logger import get_logger
from transformer.validation.quality_gates_bronze import run_quality_gates_bronze

log = get_logger("landing.convert_csv")

spark = get_spark_session("ConvertCsvToParquet")
log.info("[Landing][ConvertCSV] SparkSession iniciada.")


2025-11-12 00:03:37 [INFO] landing.convert_csv | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-12 00:03:38 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'ConvertCsvToParquet' (master=local[*]).
2025-11-12 00:03:38 [INFO] landing.convert_csv | [Landing][ConvertCSV] SparkSession iniciada.


In [ ]:
def convert_csv_to_parquet(spark: SparkSession, csv_files: list[str], stage_path: str) -> None:
    """
    Converte arquivos csv em formato parquet.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        csv_files (list[str]): Lista de caminhos dos arquivos csv a converter.
        stage_path (str): Caminho base da camada Stage.
    """
    if not csv_files:
        raise ValueError("[Landing][ConvertCSV][ERROR] Nenhum arquivo csv fornecido para conversão.")

    for csv in csv_files:
        try:
            base_name = os.path.basename(csv).replace(".csv", ".parquet")
            log.info(f"[Landing][ConvertCSV] Lendo arquivo csv: {csv}.")

            # Leitura do CSV
            df = (
                spark.read
                .option("header", True)
                .option("inferSchema", False)
                .csv(csv)
            )

            # Validação de qualidade
            if df.columns[1] == 'AIRLINE':
                required_columns = ['IATA_CODE', 'AIRLINE']
            else: 
                required_columns = ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']
            run_quality_gates_bronze(df, base_name, required_columns)

            # Escrita em parquet
            parquet_path = f"{stage_path}/{base_name}"
            df.write.mode("overwrite").option("compression", "snappy").parquet(parquet_path)
            log.info(f"[Landing][ConvertCSV] Arquivo convertido: {parquet_path}.")

        except Exception as e:
            log.error(f"[Landing][ConvertCSV][ERROR] Falha ao converter {csv}: {e}")

            raise IOError(f"Erro ao processar {csv}: {e}.") from e


In [18]:
try:
    log.info("[Landing][ConvertCSV] Iniciando job de conversão csv -> parquet.")

    csv_files = check_files_in_folder(stage_path, "*.csv")
    target_files = [f for f in csv_files if "flights_part" not in f]

    if not target_files:
        raise FileNotFoundError(f"[Landing][ConvertCSV][ERROR] Nenhum arquivo csv com o padrão encontrado em {stage_path}.")

    convert_csv_to_parquet(spark, target_files, stage_path)

    log.info(f"[Landing][ConvertCSV] Conversão concluída. {len(target_files)} arquivo(s) processado(s).")

    log.info("[Landing][ConvertCSV] Job concluído com sucesso.")

except Exception as e:
    log.exception(f"[Landing][ConvertCSV][ERROR] Falha durante execução: {e}")
    raise


2025-11-12 00:03:44 [INFO] landing.convert_csv | [Landing][ConvertCSV] Iniciando job de conversão csv -> parquet.
2025-11-12 00:03:44 [INFO] file_io | [INFO] Encontrados 12 arquivo(s).
2025-11-12 00:03:44 [INFO] landing.convert_csv | [Landing][ConvertCSV] Lendo arquivo csv: /opt/airflow/data-layer/stage/airlines.csv.
2025-11-12 00:03:44 [INFO] quality_gates_bronze | [Quality][Landing] Iniciando validações do dataset 'airlines.parquet'.
2025-11-12 00:03:45 [INFO] quality_gates_bronze | [Quality][Landing] _check_row_count_not_empty para 'airlines.parquet': OK.
2025-11-12 00:03:45 [INFO] quality_gates_bronze | [Quality][Landing] _check_schema_columns para 'airlines.parquet': OK.
2025-11-12 00:03:45 [INFO] quality_gates_bronze | [Quality][Landing] Todas as validações para 'airlines.parquet' concluídas com sucesso.
2025-11-12 00:03:46 [INFO] spark_helpers | [INFO] Dataframe salvo com sucesso em '/opt/airflow/data-layer/stage/airlines.parquet'.
2025-11-12 00:03:46 [INFO] landing.convert_csv 

In [19]:
# Encerrando a sessão Spark
spark.stop()
log.info("[Landing][ConvertCSV] Sessão Spark finalizada.")


2025-11-12 00:03:52 [INFO] landing.convert_csv | [Landing][ConvertCSV] Sessão Spark finalizada.
